In [27]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [28]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [29]:
df.shape

(569, 33)

In [30]:
df.drop(columns=['id','Unnamed: 32'],inplace=True)

In [31]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [32]:
X_train,X_test,y_train,y_test=train_test_split(df.iloc[:,1:],df.iloc[:,0],test_size=0.2)

In [33]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [34]:
#label encoding
encoder=LabelEncoder()
y_train=encoder.fit_transform(y_train)
y_test=encoder.transform(y_test)

In [35]:
#numpy array to pytorch tensor
X_train_tensor=torch.from_numpy(X_train)
X_test_tensor=torch.from_numpy(X_test)
y_train_tensor=torch.from_numpy(y_train)
y_test_tensor=torch.from_numpy(y_test)

In [36]:
X_train_tensor.shape

torch.Size([455, 30])

In [37]:
y_train_tensor.shape

torch.Size([455])

In [38]:
#define the model

class MySimpleNN():
   def __init__(self,X):
    self.weights=torch.rand(X.shape[1],1,dtype=torch.float64,requires_grad=True)
    self.bias=torch.zeros(1,dtype=torch.float64,requires_grad=True)

   def forward(self,X):
    z=torch.matmul( X, self.weights)+self.bias
    y_pred=torch.sigmoid(z)
    return y_pred

   def loss_function(self,y_pred,y):
    epsilon=1e-7
    y_pred=torch.clamp(y_pred,epsilon,1-epsilon)
    loss=-(y_train_tensor*torch.log(y_pred)+(1-y_train_tensor)*torch.log(1-y_pred)).mean()
    return loss




In [39]:
learning_rate=0.1
epochs=25

In [40]:
model=MySimpleNN(X_train_tensor)

for epoch in range(epochs):
  y_pred=model.forward(X_train_tensor)
  loss=model.loss_function(y_pred,y_train_tensor)
  loss.backward()

  with torch.no_grad():
    model.weights -= learning_rate * model.weights.grad
    model.bias -= learning_rate * model.bias.grad
  model.weights.grad.zero_()
  model.bias.grad.zero_()

  print(f'Epoch:{epoch+1},Loss:{loss.item()}')



Epoch:1,Loss:3.8065162471221727
Epoch:2,Loss:3.6908759319446296
Epoch:3,Loss:3.572216172895569
Epoch:4,Loss:3.4415104440662194
Epoch:5,Loss:3.3022661600943444
Epoch:6,Loss:3.1620971784600456
Epoch:7,Loss:3.0187162056165135
Epoch:8,Loss:2.869340947969807
Epoch:9,Loss:2.7116313122339895
Epoch:10,Loss:2.553693117702162
Epoch:11,Loss:2.391229222394694
Epoch:12,Loss:2.2278985648626364
Epoch:13,Loss:2.0633564509426687
Epoch:14,Loss:1.903552922369363
Epoch:15,Loss:1.7496383860100262
Epoch:16,Loss:1.5980286748575305
Epoch:17,Loss:1.4511621251478497
Epoch:18,Loss:1.3164660767681353
Epoch:19,Loss:1.1955570134560467
Epoch:20,Loss:1.0923515306790303
Epoch:21,Loss:1.0074186702817494
Epoch:22,Loss:0.9401122170509609
Epoch:23,Loss:0.8885422665656072
Epoch:24,Loss:0.8499454126654824
Epoch:25,Loss:0.8213164674045776


In [42]:
model.bias

tensor([-0.1095], dtype=torch.float64, requires_grad=True)

In [43]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.9).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.6015697121620178
